In [2]:
import pandas as pd
import numpy as np
movies=pd.read_csv('ml-latest-small/movies.csv')
links=pd.read_csv('ml-latest-small/links.csv')
ratings=pd.read_csv('ml-latest-small/ratings.csv')
tags=pd.read_csv('ml-latest-small/tags.csv')

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
def strrep(genre):
    return genre.replace('|',' ')

In [5]:
# x='Adventure|Animation|Children|Comedy|Fantasy'
# x.replace('|',' ')
movies['genres']=movies.genres.apply(strrep)

In [16]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action Animation Comedy Fantasy
9738,193583,No Game No Life: Zero (2017),Animation Comedy Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action Animation


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english')
genresvect=vectorizer.fit_transform(movies['genres'])

genresvect.shape

(9742, 23)

In [7]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(genresvect, genresvect)

In [8]:
cosine_sim

array([[1.        , 0.81357774, 0.15276924, ..., 0.        , 0.4210373 ,
        0.26758648],
       [0.81357774, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.15276924, 0.        , 1.        , ..., 0.        , 0.        ,
        0.57091541],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.4210373 , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.26758648, 0.        , 0.57091541, ..., 0.        , 0.        ,
        1.        ]])

In [9]:
indices=pd.Series(movies.index,index=movies['title']).drop_duplicates()
indices

title
Toy Story (1995)                                0
Jumanji (1995)                                  1
Grumpier Old Men (1995)                         2
Waiting to Exhale (1995)                        3
Father of the Bride Part II (1995)              4
                                             ... 
Black Butler: Book of the Atlantic (2017)    9737
No Game No Life: Zero (2017)                 9738
Flint (2017)                                 9739
Bungo Stray Dogs: Dead Apple (2018)          9740
Andrew Dice Clay: Dice Rules (1991)          9741
Length: 9742, dtype: int64

In [10]:
def get_recommendation(title,cosine_sim=cosine_sim):
    idx=indices[title]
    sim_scores=list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]
    print(sim_scores)

    movie_indices = [i[0] for i in sim_scores]

    return movies['title'].iloc[movie_indices]

In [11]:
get_recommendation('Toy Story (1995)')

[(1706, 0.9999999999999998), (2355, 0.9999999999999998), (2809, 0.9999999999999998), (3000, 0.9999999999999998), (3568, 0.9999999999999998), (6194, 0.9999999999999998), (6486, 0.9999999999999998), (6948, 0.9999999999999998), (7760, 0.9999999999999998), (8219, 0.9999999999999998)]


1706                                          Antz (1998)
2355                                   Toy Story 2 (1999)
2809       Adventures of Rocky and Bullwinkle, The (2000)
3000                     Emperor's New Groove, The (2000)
3568                                Monsters, Inc. (2001)
6194                                     Wild, The (2006)
6486                               Shrek the Third (2007)
6948                       Tale of Despereaux, The (2008)
7760    Asterix and the Vikings (Astérix et les Viking...
8219                                         Turbo (2013)
Name: title, dtype: object

In [ ]:
from scipy.sparse.linalg import svds
U,sigma,vt= 

In [20]:
from surprise import SVD
from surprise import Dataset,Reader
from surprise.model_selection import cross_validate
from surprise import accuracy
from surprise.model_selection import train_test_split

reader = Reader()

data=Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']],reader)
trainset, testset = train_test_split(data, test_size=.25)

# We'll use the famous SVD algorithm.
svd = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
svd.fit(trainset)
predictions = svd.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.8764


0.8764291237931479

In [21]:
from surprise import KNNBasic
knn=KNNBasic()
knn.fit(trainset)
predknn=knn.test(testset)
accuracy.rmse(predknn)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9447


0.9447151126992277

In [27]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
227,1,3744,4.0,964980694
228,1,3793,5.0,964981855
229,1,3809,4.0,964981220
230,1,4006,4.0,964982903


In [31]:
knn.predict(1, 50,5)

Prediction(uid=1, iid=50, r_ui=5, est=4.480081305548604, details={'actual_k': 40, 'was_impossible': False})

In [32]:
def hybrid(userid,title):
    idx=indices[title]
    sim_scores=list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    
    movie_indices = [i[0] for i in sim_scores]
    
    movie=movies.iloc[movie_indices]
    movie['est']=movie['movieId'].apply(lambda x: knn.predict(userid,x).est)
    movie = movie.sort_values('est', ascending=False)
    return movie.head(10)

In [35]:
hybrid(60,'Toy Story (1995)')

C:\Program Files\Anaconda\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,movieId,title,genres,est
3568,4886,"Monsters, Inc. (2001)",Adventure Animation Children Comedy Fantasy,3.967327
2355,3114,Toy Story 2 (1999),Adventure Animation Children Comedy Fantasy,3.912868
3000,4016,"Emperor's New Groove, The (2000)",Adventure Animation Children Comedy Fantasy,3.836918
6194,45074,"Wild, The (2006)",Adventure Animation Children Comedy Fantasy,3.500807
7760,91355,Asterix and the Vikings (Astérix et les Viking...,Adventure Animation Children Comedy Fantasy,3.500807
1706,2294,Antz (1998),Adventure Animation Children Comedy Fantasy,3.309552
6948,65577,"Tale of Despereaux, The (2008)",Adventure Animation Children Comedy Fantasy,3.000000
6486,53121,Shrek the Third (2007),Adventure Animation Children Comedy Fantasy,2.865559
8219,103755,Turbo (2013),Adventure Animation Children Comedy Fantasy,2.500000
2809,3754,"Adventures of Rocky and Bullwinkle, The (2000)",Adventure Animation Children Comedy Fantasy,2.356221


In [37]:
knn.predict(60, 3114)

Prediction(uid=60, iid=3114, r_ui=None, est=3.912867839578837, details={'actual_k': 40, 'was_impossible': False})